# **ManisAI**
This document provides guidance on how to use the model to run thru a testing directory to check the accuracy of the model.

### **Required Files for Testing**
1. Machine Learning Model
2. Testing Directory with Images in 8 Different Kuih Folder (Kuih Talam, Kuih Seri Muka, Kuih Ubi Kayu, Kuih Kaswi Pandan, Kuih Ketayap, Onde-onde, Kuih Lapis, Kek Lapis)
3. Class Labels File (optional)

### 1. Setting Up the Environment

Install the necessary libraries to run the model

In [ ]:
%pip uninstall -y numpy pandas tensorflow
%pip install --no-cache-dir tensorflow==2.12.0 numpy==1.23.5 pandas==1.5.3

### 2. Importing Libraries

Imports the necessary Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import torch
import os
import matplotlib.pyplot as plt
from google.colab import files
import io
import zipfile
from tqdm.notebook import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import torch.nn.functional as func
from sklearn.metrics import (
    classification_report, accuracy_score,
    roc_auc_score, precision_recall_fscore_support
)
from sklearn.preprocessing import label_binarize
import numpy as np

### 3. Upload Model

Change the 'model_filename' to where the ManisAI.pth is located

In [ ]:
model_filename = 'manisAI.pth'
model = torch.load(model_filename, map_location=torch.device('cpu'), weights_only=False)
print("Model loaded successfully!")

### Optional: Upload Label

Change the 'label_filename' to where your label class file is located

In [ ]:
labels_filename = 'labels.txt'
labels = {}
with open(labels_filename, 'r') as f:
    for line in f:
        if line.strip():
            idx, label = line.strip().split(' ', 1)
            labels[int(idx)] = label

print(f"Loaded {len(labels)} classes:")
for idx, label in labels.items():
    print(f"  {idx}: {label}")

### 4. Access Testing Directory

Change the 'test_dir' to where your testing directory is located

In [ ]:
test_dir = 'Testing/'

# Define data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_dataset = datasets.ImageFolder(root=test_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Found {len(val_dataset)} test images")

### 5. Running Predictions

Process each image for model input (resize, normalize) and make prediction

In [ ]:
predictions = []
model.eval()
class_names = val_dataset.classes

with torch.no_grad():
    for img_path in tqdm(range(len(val_dataset))):
        for inputs, labels in val_loader:
            outputs = model(inputs)

            # Apply softmax to get probabilities (along dimension 1, which is the class dimension)
            probabilities = func.softmax(outputs, dim=1)
            _, predicted = torch.max(outputs.data, 1)

            for i in range(inputs.size(0)):
                sample_probabilities = probabilities[i].tolist()
                predictions.append(
                    {
                        'predicted_class_index': predicted[i],
                        'predicted_label': class_names[predicted[i]],
                        'class_probabilities': sample_probabilities,
                        'true_class_index': labels[i],
                        'true_label': class_names[labels[i]]
                    }
                )
                print(f"{n}/{(len(val_dataset))}")

### 6. Creating Output

Convert the model's prediction results into a structured format with each images':
1. Predicted class index
2. Predicted label
3. Class probabilities
4. True class index
5. True label

In [ ]:
results_df = pd.DataFrame(predictions)
display(results_df)

### 7. Metrics Computation

Calculate the metrics of the model including:
1. Model accuracy
2. Precision, recall, and F1 for each class
3. Macro precision
4. Macro recall
5. Macro F1
6. ROC AUC per class
7. Macro ROC AUC


In [ ]:
# True and predicted labels
y_true = results_df['true_class_index'].astype(int).values
y_pred = results_df['predicted_class_index'].astype(int).values
y_probs = np.array(results_df['class_probabilities'].tolist())

## Quick fix for ROC Curve as I only have 3 classes here (DO NOT NEED THIS IF YOU HAVE 8 CLASSES IN YOUR TEST SET)
FULL_NUM_CLASSES = 8  # total number of possible classes

# Pad probability vectors to length 8
def pad_probs(probs, target_len=FULL_NUM_CLASSES):
    padded = np.zeros(target_len)
    padded[:len(probs)] = probs  # assumes probs are in order (class 0, 1, 2, ...)
    return padded

# Apply padding
y_probs_padded = np.array([pad_probs(p, FULL_NUM_CLASSES) for p in results_df['class_probabilities']])

# Update your DataFrame or use directly in metrics
y_probs = y_probs_padded

In [ ]:
# Number of classes
n_classes = FULL_NUM_CLASSES
class_names = list(range(FULL_NUM_CLASSES))

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

# Precision, Recall, F1 per class & macro
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_names, average=None)
macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

print("\n📊 Per-class metrics:")
for i, cls in enumerate(class_names):
    print(f"Class {cls}: Precision={prec[i]:.4f}, Recall={rec[i]:.4f}, F1={f1[i]:.4f}")

print(f"\n📦 Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}, Macro F1: {macro_f1:.4f}")

# ROC AUC (requires binarized labels)
y_true_bin = label_binarize(y_true, classes=class_names)

# ROC AUC per class and macro
try:
    auc_per_class = roc_auc_score(y_true_bin, y_probs, average=None, multi_class='ovr')
    auc_macro = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')

    print("\n🎯 ROC AUC per class:")
    for i, cls in enumerate(class_names):
        print(f"Class {cls}: AUC = {auc_per_class[i]:.4f}")

    print(f"\n🌐 Macro ROC AUC: {auc_macro:.4f}")

except Exception as e:
    print(f"⚠️ ROC AUC could not be computed: {e}")
